Summary: 
We have been running some A/B tests on email copy and need your help interpreting the results. The marketing team sent out two emails with different subject lines about our upcoming classes to our contacts. We randomly assigned customer segments to two groups and sent emails to 5000 members of each group. Group A received a subject line emphasizing urgency (“Hurry! Classes Are Filling Up Fast”) while Group B received a subject line emphasizing class quality (“Upcoming Classes Led by Top Experts”). Your task is to interpret the A/B test results and recommend other experiments for us to try.

Questions:
What are the results of the A/B test? What does this tell us about urgency vs quality as motivation?
How would you improve this A/B test?
What other experiments should we try to improve our email campaign performance?

In [1]:
import pandas as pd
from scipy import stats

# Create a DataFrame with the data from the CSV
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Sent": [5000, 5000],
    "Number_Opened": [3200, 3000],
    "Links_Clicked": [430, 350],
    "Classes_Purchased": [46, 32]
}

df = pd.DataFrame(data)

# Calculate rates
df["Open_Rate"] = df["Number_Opened"] / df["Number_Sent"]
df["Click_Rate"] = df["Links_Clicked"] / df["Number_Sent"]
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

print("Metrics for each group:")
print(df[["Email_Version", "Open_Rate", "Click_Rate", "Conversion_Rate"]])

# Example: Statistical significance testing for conversion rate using a z-test for proportions:
# For group A:
p1 = df.loc[0, "Conversion_Rate"]
n1 = df.loc[0, "Number_Opened"]

# For group B:
p2 = df.loc[1, "Conversion_Rate"]
n2 = df.loc[1, "Number_Opened"]

# Pooled conversion rate
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) \
              / (n1 + n2)

# Standard error for the difference:
se = (pooled_rate * (1 - pooled_rate) * (1/n1 + 1/n2)) ** 0.5

# z-score:
z_score = (p1 - p2) / se

# p-value (two-tailed test)
p_value = stats.norm.sf(abs(z_score)) * 2

print("\nZ-Test for conversion rates difference between groups:")
print("Z-Score:", z_score)
print("P-Value:", p_value)

Metrics for each group:
  Email_Version  Open_Rate  Click_Rate  Conversion_Rate
0    A. Urgency       0.64       0.086         0.014375
1    B. Quality       0.60       0.070         0.010667

Z-Test for conversion rates difference between groups:
Z-Score: 1.3092309464436958
P-Value: 0.19045613249820947


In [2]:
import pandas as pd
import plotly.express as px

# Data as provided in the CSV
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Sent": [5000, 5000],
    "Number_Opened": [3200, 3000],
    "Links_Clicked": [430, 350],
    "Classes_Purchased": [46, 32]
}

df = pd.DataFrame(data)

# Calculate the key metrics
df["Open_Rate"] = df["Number_Opened"] / df["Number_Sent"]
df["Click_Rate"] = df["Links_Clicked"] / df["Number_Sent"]
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Option 1: Grouped Bar Chart for Rates
df_rates = df[['Email_Version', 'Open_Rate', 'Click_Rate', 'Conversion_Rate']]
df_rates_long = df_rates.melt(id_vars='Email_Version', var_name='Metric', value_name='Rate')

fig1 = px.bar(
    df_rates_long,
    x='Metric',
    y='Rate',
    color='Email_Version',
    barmode='group',
    title='Email Campaign Performance Metrics (Rates)'
)
fig1.show()

# Option 2: Funnel Charts for Each Group
# Create a DataFrame representing stages of the funnel
df_funnel = pd.DataFrame({
    "Stage": ["Emails Sent", "Emails Opened", "Links Clicked", "Classes Purchased"],
    "A. Urgency": [5000, 3200, 430, 46],
    "B. Quality": [5000, 3000, 350, 32]
})

# Funnel chart for A. Urgency
fig2 = px.funnel(
    df_funnel,
    x="A. Urgency",
    y="Stage",
    title='Funnel Chart: A. Urgency'
)
fig2.show()

# Funnel chart for B. Quality
fig3 = px.funnel(
    df_funnel,
    x="B. Quality",
    y="Stage",
    title='Funnel Chart: B. Quality'
)
fig3.show()

# Option 3: Grouped Bar Chart for Count Comparisons at Each Stage
df_counts = df[['Email_Version', 'Number_Sent', 'Number_Opened', 'Links_Clicked', 'Classes_Purchased']]
df_counts_long = df_counts.melt(id_vars='Email_Version', var_name='Metric', value_name='Count')

fig4 = px.bar(
    df_counts_long,
    x='Metric',
    y='Count',
    color='Email_Version',
    barmode='group',
    title='Email Campaign Conversion Funnel (Counts per Stage)'
)
fig4.show()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [3]:
import math
import pandas as pd
import plotly.express as px
from scipy import stats

# Prepare data for only conversion metrics
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Opened": [3200, 3000],
    "Classes_Purchased": [46, 32]
}
df = pd.DataFrame(data)

# Calculate conversion rates
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Calculate standard error (SE) for each conversion rate and then the 95% confidence interval (CI)
# SE = sqrt(p*(1-p)/n) and CI = 1.96 * SE
df["SE"] = (df["Conversion_Rate"] * (1 - df["Conversion_Rate"]) / df["Number_Opened"]).apply(math.sqrt)
df["CI"] = 1.96 * df["SE"]

# Statistical significance test (z-test for proportions)
p1 = df.loc[0, "Conversion_Rate"]
n1 = df.loc[0, "Number_Opened"]
p2 = df.loc[1, "Conversion_Rate"]
n2 = df.loc[1, "Number_Opened"]

# Pooled conversion rate for the z-test:
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) / (n1 + n2)

# Standard error for the difference in conversion rates:
se_diff = math.sqrt(pooled_rate * (1 - pooled_rate) * (1/n1 + 1/n2))

# Z-score for the difference:
z_score = (p1 - p2) / se_diff

# p-value from the z-score (two-tailed test):
p_value = stats.norm.sf(abs(z_score)) * 2

# Create an annotation string based on p-value
if p_value < 0.05:
    sig_text = "Statistically Significant"
else:
    sig_text = "Not Statistically Significant"

# Create a Plotly bar chart with confidence interval error bars for conversion rates.
fig = px.bar(
    df,
    x="Email_Version",
    y="Conversion_Rate",
    error_y="CI",
    title=f'Conversion Rate Comparison (p-value = {p_value:.3f} | {sig_text})',
    labels={"Conversion_Rate": "Conversion Rate", "Email_Version": "Email Version"}
)

fig.show()

In [4]:
import pandas as pd
import plotly.graph_objects as go

# Data as provided in the CSV
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Sent": [5000, 5000],
    "Number_Opened": [3200, 3000],
    "Links_Clicked": [430, 350],
    "Classes_Purchased": [46, 32]
}

df = pd.DataFrame(data)

# Calculate key rates
df["Open_Rate"] = df["Number_Opened"] / df["Number_Sent"]
# For click rate, you might want to compare per email sent
df["Click_Rate"] = df["Links_Clicked"] / df["Number_Sent"]
# Conversion is usually defined as Classes Purchased out of those who opened the email
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Prepare data for the radar chart
categories = ["Open Rate", "Click Rate", "Conversion Rate"]
urgency_rates = [df.loc[0, "Open_Rate"], df.loc[0, "Click_Rate"], df.loc[0, "Conversion_Rate"]]
quality_rates = [df.loc[1, "Open_Rate"], df.loc[1, "Click_Rate"], df.loc[1, "Conversion_Rate"]]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=urgency_rates,
    theta=categories,
    fill='toself',
    name='A. Urgency'
))

fig.add_trace(go.Scatterpolar(
    r=quality_rates,
    theta=categories,
    fill='toself',
    name='B. Quality'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True,
    title="Radar Chart: Email Campaign Performance Metrics"
)

fig.show()

In [5]:
import plotly.graph_objects as go

# Funnel stage labels
stages = ["Emails Sent", "Emails Opened", "Links Clicked", "Classes Purchased"]

# Data for the two groups
values_urgency = [5000, 3200, 430, 46]
values_quality = [5000, 3000, 350, 32]

# Calculate the differences at each stage (A - B)
diff_values = [urg - qual for urg, qual in zip(values_urgency, values_quality)]

# For Waterfall charts, the first value is often shown as an absolute measure.
# In our case, Emails Sent are identical and then differences are computed for each subsequent stage.
measure = ["absolute", "relative", "relative", "relative"]

fig = go.Figure(go.Waterfall(
    name="Difference (A - B)",
    orientation="v",
    measure=measure,
    x=stages,
    y=diff_values,
    text=[f"{dv:+}" for dv in diff_values],
    textposition="outside",
    connector={"line": {"color": "rgb(63, 63, 63)"}}
))

fig.update_layout(
    title="Waterfall Chart: Funnel Stage Differences (A. Urgency vs. B. Quality)",
    showlegend=True
)

fig.show()

In [6]:
import math
import pandas as pd
from scipy import stats
import plotly.graph_objects as go

# Prepare data for conversion metrics
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Opened": [3200, 3000],
    "Classes_Purchased": [46, 32]
}
df = pd.DataFrame(data)

# Calculate conversion rates
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Calculate standard error (SE) and 95% CI
df["SE"] = (df["Conversion_Rate"] * (1 - df["Conversion_Rate"]) / df["Number_Opened"]).apply(math.sqrt)
df["CI"] = 1.96 * df["SE"]

# Statistical test for the difference (using a z-test for proportions)
p1, n1 = df.loc[0, "Conversion_Rate"], df.loc[0, "Number_Opened"]
p2, n2 = df.loc[1, "Conversion_Rate"], df.loc[1, "Number_Opened"]
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) / (n1 + n2)
se_diff = math.sqrt(pooled_rate * (1 - pooled_rate) * (1/n1 + 1/n2))
z_score = (p1 - p2) / se_diff
p_value = stats.norm.sf(abs(z_score)) * 2

if p_value < 0.05:
    significance = "Yes"
else:
    significance = "No"

# Create a summary table
fig = go.Figure(data=[go.Table(
    header=dict(values=["Metric", "A. Urgency", "B. Quality"],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[
        ["Conversion Rate", "95% CI", "p-value", "Stat Sig?"],
        [f"{p1:.3%}", f"{df.loc[0, 'CI']:.3%}", "", ""],
        [f"{p2:.3%}", f"{df.loc[1, 'CI']:.3%}", "", ""],
        ["", "", f"{p_value:.3f}", significance]
    ],
    fill_color='lavender',
    align='left'))
])

fig.update_layout(title="Conversion Rate Summary and Statistical Significance")
fig.show()

In [7]:
import pandas as pd
import plotly.express as px

# Data for the sunburst chart: each email version with its funnel stages
data = {
    "Email_Version": ["A. Urgency", "A. Urgency", "A. Urgency", "A. Urgency",
                      "B. Quality", "B. Quality", "B. Quality", "B. Quality"],
    "Stage": ["Emails Sent", "Emails Opened", "Links Clicked", "Classes Purchased"] * 2,
    "Count": [5000, 3200, 430, 46, 5000, 3000, 350, 32]
}

df_sunburst = pd.DataFrame(data)

fig = px.sunburst(
    df_sunburst,
    path=['Email_Version', 'Stage'],
    values='Count',
    title="Sunburst Chart of Email Campaign Funnel by Email Version"
)
fig.show()

In [8]:
import math
import pandas as pd
import plotly.express as px
from scipy import stats

# Data for conversion metrics
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Opened": [3200, 3000],
    "Classes_Purchased": [46, 32]
}
df = pd.DataFrame(data)

# Calculate conversion rates
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Calculate standard error (SE) and 95% confidence interval (CI)
# SE = sqrt(p*(1-p)/n) and CI = 1.96 * SE
df["SE"] = (df["Conversion_Rate"] * (1 - df["Conversion_Rate"]) / df["Number_Opened"]).apply(math.sqrt)
df["CI"] = 1.96 * df["SE"]

# Perform a z-test for the conversion rate difference
p1 = df.loc[0, "Conversion_Rate"]
n1 = df.loc[0, "Number_Opened"]
p2 = df.loc[1, "Conversion_Rate"]
n2 = df.loc[1, "Number_Opened"]

# Pooled conversion rate
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) / (n1 + n2)
# Standard error for the difference in conversion rates
se_diff = math.sqrt(pooled_rate * (1 - pooled_rate) * (1/n1 + 1/n2))
# Z-score and p-value (two-tailed test)
z_score = (p1 - p2) / se_diff
p_value = stats.norm.sf(abs(z_score)) * 2

# Determine significance message based on the p-value
if p_value < 0.05:
    sig_text = "Statistically Significant"
else:
    sig_text = "Not Statistically Significant"

# Create the Plotly bar chart with error bars
fig = px.bar(
    df,
    x="Email_Version",
    y="Conversion_Rate",
    error_y="CI",
    title=f'Conversion Rate Comparison (p-value = {p_value:.3f} | {sig_text})',
    labels={"Conversion_Rate": "Conversion Rate", "Email_Version": "Email Version"}
)
fig.show()

# Build a textual conclusion based on our findings
print("----- CONCLUSION -----")
if p_value < 0.05:
    if p1 > p2:
        print("The A/B test indicates that the 'A. Urgency' email version has a significantly higher conversion rate than the 'B. Quality' version.")
        print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
        print(f"'B. Quality' Conversion Rate: {p2:.2%}")
    elif p2 > p1:
        print("The A/B test indicates that the 'B. Quality' email version has a significantly higher conversion rate than the 'A. Urgency' version.")
        print(f"'B. Quality' Conversion Rate: {p2:.2%}")
        print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
else:
    print("The difference in conversion rates between 'A. Urgency' and 'B. Quality' is not statistically significant.")
    print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
    print(f"'B. Quality' Conversion Rate: {p2:.2%}")

print("----------------------")

----- CONCLUSION -----
The difference in conversion rates between 'A. Urgency' and 'B. Quality' is not statistically significant.
'A. Urgency' Conversion Rate: 1.44%
'B. Quality' Conversion Rate: 1.07%
----------------------


In [9]:
import math
import pandas as pd
import plotly.express as px
from scipy import stats

# Data for conversion metrics
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Opened": [3200, 3000],
    "Classes_Purchased": [46, 32]
}
df = pd.DataFrame(data)

# Calculate conversion rates
df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Opened"]

# Calculate standard error (SE) and 95% confidence interval (CI)
df["SE"] = (df["Conversion_Rate"] * (1 - df["Conversion_Rate"]) / df["Number_Opened"]).apply(math.sqrt)
df["CI"] = 1.96 * df["SE"]

# Perform a z-test for the conversion rate difference
p1 = df.loc[0, "Conversion_Rate"]
n1 = df.loc[0, "Number_Opened"]
p2 = df.loc[1, "Conversion_Rate"]
n2 = df.loc[1, "Number_Opened"]

# Compute the pooled conversion rate and standard error for the difference
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) / (n1 + n2)
se_diff = math.sqrt(pooled_rate * (1 - pooled_rate) * (1/n1 + 1/n2))
z_score = (p1 - p2) / se_diff

# Two-tailed p-value
p_value = stats.norm.sf(abs(z_score)) * 2

# Determine significance message based on the p-value
if p_value < 0.05:
    sig_text = "Statistically Significant"
else:
    sig_text = "Not Statistically Significant"

# Create the Plotly bar chart with error bars
fig = px.bar(
    df,
    x="Email_Version",
    y="Conversion_Rate",
    error_y="CI",
    title=f'Conversion Rate Comparison (p-value = {p_value:.3f} | {sig_text})',
    labels={"Conversion_Rate": "Conversion Rate", "Email_Version": "Email Version"}
)
fig.show()

# Build a textual conclusion based on our findings
print("----- CONCLUSION -----")
if p_value < 0.05:
    if p1 > p2:
        print("The A/B test indicates that the 'A. Urgency' email version has a significantly higher conversion rate than the 'B. Quality' version.")
        print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
        print(f"'B. Quality' Conversion Rate: {p2:.2%}")
    elif p2 > p1:
        print("The A/B test indicates that the 'B. Quality' email version has a significantly higher conversion rate than the 'A. Urgency' version.")
        print(f"'B. Quality' Conversion Rate: {p2:.2%}")
        print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
else:
    print("The difference in conversion rates between 'A. Urgency' and 'B. Quality' is not statistically significant.")
    print(f"'A. Urgency' Conversion Rate: {p1:.2%}")
    print(f"'B. Quality' Conversion Rate: {p2:.2%}")
    print("\nExplanation:")
    print("Since the p-value is greater than 0.05 (the typical significance level),")
    print("the observed difference between the two conversion rates could be due to random chance.")
    print("This means that our data does not provide strong enough evidence to conclude that")
    print("one email version outperforms the other in a statistically significant way.")
    
print("----------------------")

----- CONCLUSION -----
The difference in conversion rates between 'A. Urgency' and 'B. Quality' is not statistically significant.
'A. Urgency' Conversion Rate: 1.44%
'B. Quality' Conversion Rate: 1.07%

Explanation:
Since the p-value is greater than 0.05 (the typical significance level),
the observed difference between the two conversion rates could be due to random chance.
This means that our data does not provide strong enough evidence to conclude that
one email version outperforms the other in a statistically significant way.
----------------------


In [10]:
import math
import pandas as pd
from scipy import stats
import plotly.express as px

# Data as provided in the CSV
data = {
    "Email_Version": ["A. Urgency", "B. Quality"],
    "Number_Sent": [5000, 5000],
    "Number_Opened": [3200, 3000],
    "Classes_Purchased": [46, 32]
}
df = pd.DataFrame(data)

# Calculate conversion rates per email sent
df["Conversion_Rate_Sent"] = df["Classes_Purchased"] / df["Number_Sent"]

# For this test, our "n" becomes Number Sent for each version
# Calculate standard error (SE) and 95% confidence intervals
df["SE"] = ((df["Conversion_Rate_Sent"] * (1 - df["Conversion_Rate_Sent"])) / df["Number_Sent"]).apply(math.sqrt)
df["CI"] = 1.96 * df["SE"]

# Perform a z-test for the difference in conversion rates per email sent
p1 = df.loc[0, "Conversion_Rate_Sent"]
n1 = df.loc[0, "Number_Sent"]
p2 = df.loc[1, "Conversion_Rate_Sent"]
n2 = df.loc[1, "Number_Sent"]

# Pooled conversion rate for per email sent
pooled_rate = (df.loc[0, "Classes_Purchased"] + df.loc[1, "Classes_Purchased"]) / (n1 + n2)
se_diff = ((pooled_rate * (1 - pooled_rate)) * (1/n1 + 1/n2)) ** 0.5
z_score = (p1 - p2) / se_diff
p_value = stats.norm.sf(abs(z_score)) * 2

# Interpretation for statistical significance
if p_value < 0.05:
    sig_text = "Statistically Significant"
else:
    sig_text = "Not Statistically Significant"

# Create the Plotly bar chart with error bars
fig = px.bar(
    df,
    x="Email_Version",
    y="Conversion_Rate_Sent",
    error_y="CI",
    title=f'Conversion Rate (per Email Sent) Comparison (p-value = {p_value:.3f} | {sig_text})',
    labels={"Conversion_Rate_Sent": "Conversion Rate per Email Sent", "Email_Version": "Email Version"}
)
fig.show()

# Textual conclusion
print("----- CONCLUSION -----")
if p_value < 0.05:
    if p1 > p2:
        print("The overall campaign shows that 'A. Urgency' has a significantly higher conversion rate per email sent.")
    else:
        print("The overall campaign shows that 'B. Quality' has a significantly higher conversion rate per email sent.")
else:
    print("The difference in conversion rates per email sent between the two email versions is not statistically significant.")
    print("This suggests that when taking the entire funnel into account (from sent to conversion),")
    print("the observed differences might be due to random chance or small influences from both open and conversion behavior.")
print("----------------------")

----- CONCLUSION -----
The difference in conversion rates per email sent between the two email versions is not statistically significant.
This suggests that when taking the entire funnel into account (from sent to conversion),
the observed differences might be due to random chance or small influences from both open and conversion behavior.
----------------------


In [1]:
   import pandas as pd
   import plotly.express as px

   # Here we create the data manually as extracted from the CSV
   data = {
       "Email_Version": ["A. Urgency", "B. Quality"],
       "Number_Sent": [5000, 5000],
       "Number_Opened": [3200, 3000],
       "Links_Clicked": [430, 350],
       "Classes_Purchased": [46, 32]
   }
   df = pd.DataFrame(data)

   # Calculate rates (using Number Sent for open and click rate; you can also compute conversion from opened if desired)
   df["Open_Rate"] = df["Number_Opened"] / df["Number_Sent"]
   df["Click_Rate"] = df["Links_Clicked"] / df["Number_Sent"]
   df["Conversion_Rate"] = df["Classes_Purchased"] / df["Number_Sent"]

   # Melt the DataFrame for a grouped bar chart format
   df_long = df.melt(id_vars="Email_Version", 
                     value_vars=["Open_Rate", "Click_Rate", "Conversion_Rate"],
                     var_name="Metric", 
                     value_name="Rate")

   # Create the grouped bar chart
   fig = px.bar(
       df_long,
       x="Metric",
       y="Rate",
       color="Email_Version",
       barmode="group",
       title="Key Email Campaign Metrics Comparison"
   )
   fig.show()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
